#### 양자화된 모델에 대한 문장 생성

1. casllm-base-7b-hf: Meta Llama2 7B 허깅페이스 모델
2. casllm-base-7b-hf-chat: Meta Llama2 7B-Chat 허깅페이스 모델
4. Llama2 모델은 GPU T4x16G 에서는 사용할 수 없다. 공식문서에서는 T4x27G 이상 사용을 권하고 있다.
4. Llama2을 변환한 huggingface 모델은 가중치가 float 32 이다. (First parameter data type: torch.float32)


In [ ]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer

In [ ]:
model_id = "/data/bwllm/models/casllm-base-7b-hf"
#model_id = "/data/bwllm/models/casllm-base-7b-hf-chat"

model = LlamaForCausalLM.from_pretrained(model_id)
tokenizer = LlamaTokenizer.from_pretrained(model_id)

```
# CUDA(GPU)를 사용할 수 있는지 확인
# 그런데 T4 16G 에서는 아래의 에러가 발생하였다.
# CUDA out of memory. Tried to allocate 172.00 MiB. GPU 0 has a total capacity of 14.58 GiB of which 160.44 MiB is free
# 결론적으로 T4 16G 에서 GPU는 사용할 수 없다.
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("GPU is not available. Using CPU.")

# 모델을 해당 디바이스로 이동
model.to(device)
```

In [ ]:
# 모델의 첫 번째 파라미터의 데이터 타입 확인
first_param_dtype = next(model.parameters()).dtype
print(f"First parameter data type: {first_param_dtype}")

In [ ]:
# 양자화 여부 확인
if 'quantized' in str(first_param_dtype):
    print("The model is quantized.")
else:
    print("The model is not quantized.")

In [ ]:
print(model)

In [ ]:
def make_answer(prompt_status, question):
    
    messages = prompt_status + [ {"role": "질문", "content": question}]    
    prompt_text = "\n".join(
        [f"### {msg['role']}:\n{msg['content']}" for msg in messages]
    )
    return prompt_text
        
        

In [ ]:
prompt_status = [
            {
                "role": "맥락",
                "content": "KoAlpaca(코알파카)는 EleutherAI에서 개발한 Polyglot-ko 라는 한국어 모델을 기반으로, 자연어 처리 연구자 Beomi가 개발한 모델입니다.",
            },
            {
                "role": "맥락",
                "content": "ChatKoAlpaca(챗코알파카)는 KoAlpaca를 채팅형으로 만든 것입니다.",
            },
            {"role": "명령어", "content": "친절한 AI 챗봇인 ChatKoAlpaca 로서 답변을 합니다."},
            {
                "role": "명령어",
                "content": "인사에는 짧고 간단한 친절한 인사로 답하고, 아래 대화에 간단하고 짧게 답해주세요.",
            },
        ]

prompt_status2 = []

question = "정보화시스템에서의 감리절차에 대해 알려줘"
prompt_text = make_answer(prompt_status2, question)

print(prompt_text)

In [ ]:
# 토크나이저를 사용해 입력 텍스트 인코딩
inputs = tokenizer.encode(prompt_text, return_tensors='pt')

In [ ]:
# 모델을 GPU로 이동
# T4 16G 에서 메모리에 로딩하는 것을 실패함.
#model.to('cuda')

# 입력 데이터도 GPU로 이동
#inputs = inputs.to('cuda')

print(inputs)

In [ ]:
# 모델을 사용해 문장 생성
# num_return_sequences는 반환할 문장 수, max_length는 최대 길이를 설정
output_sequences = model.generate(
    input_ids=inputs,
    max_length=2000,
    num_return_sequences=1,
    temperature=1.0,  # 생성 다양성 조절
)

In [ ]:
print(output_sequences)

In [ ]:
# 생성된 문장을 디코딩하여 출력
generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
print(generated_text)